# Parsing shoe compositions

The goal of this notebook is to look at some practical NLP code, understand the concept of caching, and get familiar with working with JSON objects. We'll be using the data provided in [this](https://maxhalford.github.io/blog/carbonfact-nlp-open-problem/) blog post.

## The data

Let's start looking at the data.

In [1]:
import pathlib

inputs = pathlib.Path('shoes/inputs.txt').read_text().splitlines()
len(inputs)

600

In [2]:
import json

outputs = json.loads(pathlib.Path('shoes/outputs.json').read_text())
len(outputs)

600

Now let's write a little script to print out a random sample. This will help us in getting familiar with the parsing task.

In [3]:
import random

i = random.randint(0, len(inputs))
print("⬇️ INPUT")
print()
print(inputs[i])
print()
print("⬆️ OUTPUT")
print()
print(json.dumps(outputs[i], indent=4, sort_keys=True))

⬇️ INPUT

52%polyamide,39%cotton,9%elastane

⬆️ OUTPUT

{
    "": [
        {
            "material": "polyamide",
            "proportion": 52.0
        },
        {
            "material": "cotton",
            "proportion": 39.0
        },
        {
            "material": "elastane",
            "proportion": 9.0
        }
    ]
}


## A first parser

Let's attempt to write a first parser. We'll be using the [`regex` library](https://github.com/mrabarnett/mrab-regex), which is not part of the standard library. You'll have to install it:

```sh
pip install regex
```

In [4]:
import regex

def named_pattern(name, pattern):
    return f"(?P<{name}>{pattern})"


def many(pattern, at_least_one=True):
    return f"({pattern})+" if at_least_one else f"({pattern})*"


def sep(pattern, sep):
    return pattern + many(sep + pattern, at_least_one=False)


def split_composition_into_components(text):
    """

    >>> split_composition_into_components("Upper: 80% Leather, 20% Textile")
    {'Upper': '80% Leather, 20% Textile'}

    """
    component = ""
    materials = []
    component_materials = {}

    for token in regex.split(r"\s+", text):
        if token.endswith(":"):
            if materials:
                component_materials[component] = " ".join(materials)
                materials = []
            component = token.rstrip(":")
        else:
            materials.append(token)
    else:
        if materials:
            component_materials[component] = " ".join(materials)

    return component_materials


def parse_materials(text):
    """

    >>> parse_materials("80% Leather 20% Textile")
    [{'material': 'Leather', 'proportion': 80.0}, {'material': 'Textile', 'proportion': 20.0}]

    """
    material_pat = named_pattern("material", r"[a-zA-ZÀ-ÿ\-\s']+[a-zA-ZÀ-ÿ\-']")
    proportion_pat = named_pattern("proportion", r"\d{1,3}([,\.]\d{1,2})?") + "%?"

    pattern = sep(rf"{proportion_pat}\s*{material_pat}", " ")
    match = regex.match(pattern, text)

    if not match:
        return []

    return [
        {
            "material": m,
            "proportion": float(p)
        }
        for m, p in zip(
            match.capturesdict()["material"],
            match.capturesdict()["proportion"],
        )
    ]


def parse_composition(text):
    component_materials = split_composition_into_components(text)
    return {
        component: parse_materials(materials)
        for component, materials in component_materials.items()
    }


parsings = []
for inp in inputs:
    parsing = parse_composition(inp)
    parsings.append(parsing)

In [5]:
def performance_report(parsings, outputs):
    n_correct = sum(parsing == output for parsing, output in zip(parsings, outputs))
    return f"{n_correct} correct out of {len(parsings)} ({n_correct / len(parsings) * 100:.2f}%)"

print(performance_report(parsings, outputs))

132 correct out of 600 (22.00%)


**🤔 Question: even though this first parser is not great, what would you say are its pros?**

👀 Sidebar: have you noticed the comments under some of the functions? These are [docstrings](https://realpython.com/documenting-python-code/). In particular, these documents are using `>>>`, which is indicative of [doctests](https://docs.python.org/3/library/doctest.html). These are code comments which act as documentation by showing how the function can be used. These lines of code can also be tested, as so: 

In [6]:
import doctest

doctest.testmod()

TestResults(failed=0, attempted=2)

From the command-line, you could also execute these doctests with `pytest`:

```sh
pytest --doctest-modules
```

## Looking at false positives

Anyway, back to our parsing task. The first thing we should is look at false positives: samples where the parsing was incorrect. The more false positives we look at, the more we'll understand where our parser is going wrong.

In [7]:
incorrect_parsings = [
    (inp, parsing, output)
    for inp, parsing, output in zip(inputs, parsings, outputs)
    if parsing != output
]

i = random.randint(0, len(incorrect_parsings))
inp, parsing, output = incorrect_parsings[i]
print("⬇️ INPUT")
print()
print(inp)
print()
print("❌ PARSING")
print()
print(json.dumps(parsing, indent=4, sort_keys=True))
print()
print("✅ EXPECTED OUTPUT")
print()
print(json.dumps(output, indent=4, sort_keys=True))

⬇️ INPUT

72%polyamide,19%cotton,9%elastane

❌ PARSING

{
    "": [
        {
            "material": "polyamide",
            "proportion": 72.0
        }
    ]
}

✅ EXPECTED OUTPUT

{
    "": [
        {
            "material": "polyamide",
            "proportion": 72.0
        },
        {
            "material": "cotton",
            "proportion": 19.0
        },
        {
            "material": "elastane",
            "proportion": 9.0
        }
    ]
}


## Improving the parser

After looking at a few cases, it seems that `parse_materials` is only capturing the first material in some cases. This is because it is assuming the materials are separated with a blank space, but doesn't handle commas. Let's fix that. Usually, our code would be in a script, and we would edit it in place. For this tutorial, we'll just copy/paste the `parse_materials` function and edit it here. It's worth spending some time thinking about how you would this set this up for yourself.

In [8]:
def parse_materials(text):
    material_pat = named_pattern("material", r"[a-zA-ZÀ-ÿ\-\s']+[a-zA-ZÀ-ÿ\-']")
    proportion_pat = named_pattern("proportion", r"\d{1,3}([,\.]\d{1,2})?") + "%?"

    pattern = sep(rf"{proportion_pat}\s*{material_pat}", "[,\s]")
    match = regex.match(pattern, text)

    if not match:
        return []

    return [
        {
            "material": m,
            "proportion": float(p)
        }
        for m, p in zip(
            match.capturesdict()["material"],
            match.capturesdict()["proportion"],
        )
    ]

Let's reparse all the inputs and check the new performance. 

In [9]:
parsings = []
for inp in inputs:
    parsing = parse_composition(inp)
    parsings.append(parsing)

print(performance_report(parsings, outputs))

306 correct out of 600 (51.00%)


**🤔 Question: we reparsed all the inputs, not only the false positives. Why is that important?**

Parsing tasks are very common in the data science world. Indeed, a data scientist's job usually starts by ingesting data from several sources. It's often the case that the data is messy and needs scrubbing. Parsing structured data from text is a typical task to perform. Having a good setup where you can change code and get quick feedback about the impact of said change is a game-changer. This is referred to as a "human-in-the-loop" setup.

## Online processing

The dataset is quite small, it only contains 600 cases. NLP datasets can be much larger. Let's talk a little bit about we would handle a larger corpus.

The first thing we can do is online processing. Our parsing logic is a pure function which doesn't have to be trained on a dataset. In other words, it's unsupervised. What this entails is that the parsing function can be applied to each input individually. In other words, we don't need all the inputs loaded in memory.

In [10]:
parsings = []

with open('shoes/inputs.txt') as inputs:
    for inp in inputs:
        parsing = parse_composition(inp)
        parsings.append(parsing)

print(performance_report(parsings, outputs))

306 correct out of 600 (51.00%)


We got exactly the same performance, which is expected. The difference is that we could handle large datasets without clogging the RAM.

Actually, our logic could still be improved. What we just did in the previous cell is great in that we read the inputs one by one. However, we are storing the parsings in a list which grows with time. We are also using `outputs`, which is list stored in memory. Ideally, we should be measuring the performance online too.

One of the issues with the JSON file format is that it is not made for reading one entry at a time. That's a pretty big con, considering that JSON is ubiquitous in the machine learning world. Thankfully, there are other formats which provide the best of both worlds. For instance, there is the [JSON Lines](https://jsonlines.org/) format, which simply stores each entry in a JSON array on a row.

You could say that JSON Lines isn't a new format; it's more of a convention. Nowadays, large NLP datasets are often provided as `.jsonl` files. In our case, we'll have to build that file ourselves. There are Python libraries to make this process easier -- e.g. [`jsonlines`](https://jsonlines.readthedocs.io/en/latest/) -- but it's pretty straightforward to work with without extra dependencies.

In [13]:
with open('shoes/outputs.jsonl', 'w') as f:
    for output in outputs:
        f.write(json.dumps(output) + '\n')

In [14]:
!head -n 3 shoes/outputs.jsonl

{"lace": [{"material": "nylon", "proportion": 88.0}, {"material": "spandex", "proportion": 12.0}], "string": [{"material": "nylon", "proportion": 88.0}, {"material": "spandex", "proportion": 12.0}], "top_body": [{"material": "polyester", "proportion": 100.0}]}
{"": [{"material": "polyester", "proportion": 92.0}, {"material": "spandex", "proportion": 8.0}]}
{"": [{"material": "rayon", "proportion": 95.0}, {"material": "spandex", "proportion": 5.0}]}


Now we can refactor our parsing logic to do everything online.

In [16]:
with (
    open('shoes/inputs.txt') as inputs_stream,
    open('shoes/outputs.jsonl') as outputs_stream   
):
    n_samples = 0
    n_correct = 0
    for inp, output in zip(inputs_stream, outputs_stream):
        n_samples += 1 
        output = json.loads(output)
        parsing = parse_composition(inp)
        n_correct += parsing == output

print(f"{n_correct} correct out of {n_samples} ({n_correct / n_samples * 100:.2f}%)")

306 correct out of 600 (51.00%)


This script consumes the smallest amount of memory possible. The parsing as well as the performance tracking are done online.

**🤔 Question: if we wanted to improve our parsing logic, what issue would we now face?**

## Caching

The dataset is quite small. Regardless of how we process it, the results can be obtained quite fast. But what if the dataset were much larger? Let's artificially increase the size of the input and output files. We'll do that by processing them several times.

In [18]:
n_samples = 0
n_correct = 0

for _ in range(1000):
    with (
        open('shoes/inputs.txt') as inputs_stream,
        open('shoes/outputs.jsonl') as outputs_stream   
    ):
        for inp, output in zip(inputs_stream, outputs_stream):
            n_samples += 1 
            output = json.loads(output)
            parsing = parse_composition(inp)
            n_correct += parsing == output

print(f"{n_correct} correct out of {n_samples} ({n_correct / n_samples * 100:.2f}%)")

306000 correct out of 600000 (51.00%)


It's normal that this is taking longer. Processing time should be directly correlated with the amount of data. However, the above case is an illustration of repetitive computation. Each time we loop on a file, we repeat the same computation.

A nice trick to know about is caching. The idea is simple: store the output after processing each input, and check if an input has already been processed before processing it.

In [22]:
import functools

@functools.cache
def cached_parse_composition(text):
    return parse_composition(text)

n_samples = 0
n_correct = 0

for _ in range(1000):
    with (
        open('shoes/inputs.txt') as inputs_stream,
        open('shoes/outputs.jsonl') as outputs_stream   
    ):
        for inp, output in zip(inputs_stream, outputs_stream):
            n_samples += 1 
            output = json.loads(output)
            parsing = cached_parse_composition(inp)
            n_correct += parsing == output

print(f"{n_correct} correct out of {n_samples} ({n_correct / n_samples * 100:.2f}%)")

306000 correct out of 600000 (51.00%)


Much faster! By default, `functools.cache` caches all the results. However, there are options to set a memory budget if necessary.

Caching is one of the oldest tricks in the book: don't repeat what you already know.

## JSON handling

To conclude this tutorial, let's talk a little about handling JSON data.

In [23]:
outputs = json.loads(pathlib.Path('shoes/outputs.json').read_text())

Let's say we wanted to find the top material for each component. We could do that with some Python logic.

In [73]:
component_counts = collections.defaultdict(collections.Counter)

for output in outputs:
    for component, materials in output.items():
        for material in materials:
            component_counts[component][material['material']] += 1

for component, counts in component_counts.items():
    material, count = counts.most_common(1)[0]
    print(f"{component} 👉 {material} ({count})")

lace 👉 nylon (82)
string 👉 nylon (2)
top_body 👉 polyester (5)
 👉 polyamide (185)
mesh 👉 elastane (42)
body_panty 👉 nylon (1)
body 👉 spandex (83)
fabric 👉 elastane (25)
forro 👉 polyester (2)
ruffle 👉 polyester (1)
elastic 👉 polyester (9)
bottom 👉 polyester (8)
top 👉 spandex (8)
pant 👉 spandex (6)
tank 👉 spandex (4)
lining 👉 polyester (9)
micro 👉 elastane (6)
marl_fabric 👉 polyester (1)
crochet 👉 polyester (2)
liner 👉 polyester (3)
panty 👉 nylon (1)
edge_lace 👉 nylon (2)
centre_front_and_wings 👉 polyamide (1)
cup_lining 👉 polyester (2)
cup_shell 👉 polyamide (1)
back_panel 👉 polyamide (1)
front_panel 👉 polyamide (1)
cami 👉 polyester (2)
short 👉 rayon (2)
gusset 👉 cotton (4)
g-string 👉 polyamide (6)
pants 👉 polyester knitted (1)
rib 👉 cotton (3)
aol 👉 polyamide (2)
shell 👉 polyester (1)
ank 👉 rayon (1)
knitted_top 👉 cotton woven top (1)
trim_lace 👉 polyamide (1)
striped_mesh 👉 polyamide (1)


This was of doing analysis is very imperative. It's probably easy to follow the logic, but it's not the canonical way of doing data analysis. A good idea is to convert the data to a table, allowing us to use a grammar with which we're more familiar.

If you're lucky with the layout of the JSON file, you can use `pd.json_normalize` or `pd.read_json` with default parameters. In our case, we have to a little bit of work.

In [74]:
import pandas as pd

shoes = pd.json_normalize(
    [
        {
            'shoe_id': i,
            'component': component,
            'materials': materials
        }
        for i, output in enumerate(outputs)
        for component, materials in output.items()
    ],
    record_path='materials',
    meta=['shoe_id', 'component']
)
shoes.head()

,material,proportion,shoe_id,component
0,nylon,88.0,0,lace
1,spandex,12.0,0,lace
2,nylon,88.0,0,string
3,spandex,12.0,0,string
4,polyester,100.0,0,top_body


Now we can do our analysis with pandas.

In [72]:
(
    shoes
    .groupby(['component', 'material'])
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
    .groupby('component')
    .head(1)
)

,component,material,count
9,,polyamide,185
35,body,spandex,83
75,lace,nylon,82
89,mesh,elastane,42
56,fabric,elastane,25
55,elastic,polyester,9
84,lining,polyester,9
39,bottom,polyester,8
121,top,spandex,8
68,g-string,polyamide,6
